## Generate the ANN parameters

The MOM6 ANN module reads data in a very specific format. 

There are 3 parameters that are set: 
- USE_ANN - this is a logical, which needs to be set in MOM_parameter/ MOM_override. It can be .true. or .false.
- ANN_num_layers - This the number of layers in the model (including the input and output layers) 
- ANN_PARAMS_FILE - The name of the nextcdf file, which contains all the weights, biases, normalization constants etc. 

##### The ANN_PARAMS_FILE

This netcdf file has a number of variables. 
- `layer_sizes` - An integer array, with size equal to ANN_num_layers. Each integer corresponds to the number of nodes in that layers. 
- `An`, `bn` - These are weights and bias matrices. n ranges from 0 onwards. The number of these parameter sets will be equal to the (number of layers -1); this is because each set of parameters transforms nodes of one layer to the next.
- normalizations as `input_norms`, `output_norms`

In [1]:
import numpy as np
import xarray as xr

The code that writes this file looks a bit like:

```
 def save_weights_nc_windowed(self, nc_fname): 
        
        input_norms = np.zeros((len(self.input_channels),))
        output_norms = np.zeros((len(self.output_channels),))

        for n, i in enumerate(self.input_channels):
            input_norms[n] = self.ML_data.norm_factors[i].values

        for n, i in enumerate(self.output_channels):
            output_norms[n] = self.ML_data.norm_factors[i].values

        ds_layers = xr.Dataset()

        ds_layers['layer_sizes'] = xr.DataArray(np.array([len(self.input_channels)*self.ML_data.window_size**2,
                                                          self.network.shape[0], 
                                                          self.network.shape[1], len(self.output_channels)]).astype('int32'), 
                                                dims=['num_layers'])

        ds_layers['A0'] = xr.DataArray(np.array(self.state.params['params']['layers_0']['kernel']).astype('float32'), dims=['input', 'layer1'])
        ds_layers['A1'] = xr.DataArray(np.array(self.state.params['params']['layers_1']['kernel']).astype('float32'), dims=['layer1', 'layer2'])
        ds_layers['A2'] = xr.DataArray(np.array(self.state.params['params']['layers_2']['kernel']).astype('float32'), dims=['layer2', 'output'])

        if self.network.bias:
            ds_layers['b0'] = xr.DataArray(np.array(self.state.params['params']['layers_0']['bias']).astype('float32'), dims=['layer1'])
            ds_layers['b1'] = xr.DataArray(np.array(self.state.params['params']['layers_1']['bias']).astype('float32'), dims=['layer2'])
            ds_layers['b2'] = xr.DataArray(np.array(self.state.params['params']['layers_2']['bias']).astype('float32'), dims=['output'])
        else: 
            ds_layers['b0'] = xr.DataArray(np.zeros(self.network.shape[0]).astype('float32'), dims=['layer1'])
            ds_layers['b1'] = xr.DataArray(np.zeros(self.network.shape[1]).astype('float32'), dims=['layer2'])
            ds_layers['b2'] = xr.DataArray(np.zeros(self.network.shape[2]).astype('float32'), dims=['output'])


        ds_layers['input_norms'] = xr.DataArray(input_norms.astype('float32'), dims=['input_channels'])
        ds_layers['output_norms'] = xr.DataArray(output_norms.astype('float32'), dims=['output'])
        
        ds_layers.attrs['CKPT_DIR'] = self.CKPT_DIR
        ds_layers.attrs['shape'] = self.network.shape
        ds_layers.attrs['num_in'] = len(self.input_channels)
        ds_layers.attrs['input_channels'] = self.input_channels
        ds_layers.attrs['output_channels'] = self.output_channels
        

        ds_layers.to_netcdf(nc_fname, mode='w')

```

### Diffusive case

For testing we pass in some A and b such that the output would be like a thickness diffusion model. 

In [2]:
num_layers = 2 # input slopes and output KS

ds = xr.Dataset()

ds['num_layers'] = np.int32(num_layers)

ds['layer_sizes'] = xr.DataArray(np.array([2,2]).astype('int32'), dims=['layer_num'])

ds['A0'] = xr.DataArray(np.array([[-1000,0], [0, -1000]]).astype('float32'), dims=['input', 'output'])
ds['b0'] = xr.DataArray(np.array([0,0]).astype('float32'), dims=['output'])

ds['input_norms'] = xr.DataArray(np.array([1,1]).astype('float32'), dims=['input'])
ds['output_norms'] = xr.DataArray(np.array([1,1]).astype('float32'), dims=['output'])

In [3]:
ds

<xarray.Dataset> Size: 52B
Dimensions:       (layer_num: 2, input: 2, output: 2)
Dimensions without coordinates: layer_num, input, output
Data variables:
    num_layers    int32 4B 2
    layer_sizes   (layer_num) int32 8B 2 2
    A0            (input, output) float32 16B -1e+03 0.0 0.0 -1e+03
    b0            (output) float32 8B 0.0 0.0
    input_norms   (input) float32 8B 1.0 1.0
    output_norms  (output) float32 8B 1.0 1.0

In [4]:
ds.num_layers

<xarray.DataArray 'num_layers' ()> Size: 4B
array(2, dtype=int32)

In [17]:
ds.to_netcdf('./rho_flux_ann_params.nc', mode='w')

In [17]:
ds.to_netcdf('/scratch/db194/mom6/tests_Phillips_2layer/compare_GM_2_ANN/Phillips_2layer_20km_ANN/ann_params.nc', mode='w')

## Read in a trained param set to check

In [2]:
ds_rho = xr.open_dataset('./ann_instance_6Nov.nc')

In [3]:
ds_rho

<xarray.Dataset> Size: 11kB
Dimensions:       (dummy_dimension: 1, nlayers: 4, ncol0: 45, nrow0: 32,
                   ncol1: 32, nrow1: 32, ncol2: 32, nrow2: 2)
Dimensions without coordinates: dummy_dimension, nlayers, ncol0, nrow0, ncol1,
                                nrow1, ncol2, nrow2
Data variables:
    num_layers    (dummy_dimension) int32 4B ...
    layer_sizes   (nlayers) int32 16B ...
    A0            (ncol0, nrow0) float32 6kB ...
    b0            (nrow0) float32 128B ...
    A1            (ncol1, nrow1) float32 4kB ...
    b1            (nrow1) float32 128B ...
    A2            (ncol2, nrow2) float32 256B ...
    b2            (nrow2) float32 8B ...
    x_test        (ncol0) float32 180B ...
    y_test        (nrow2) float32 8B ...
    input_norms   (ncol0) float32 180B ...
    output_norms  (nrow2) float32 8B ...

### Arbitrary test case 

In [27]:
ds = xr.Dataset()

In [39]:
ds['layer_sizes'] = xr.DataArray(np.array([2,4,4,2]).astype('int32'), dims=['num_layers'])

In [90]:
np.random.seed(0)
ds['A0'] = xr.DataArray(np.random.randn(ds.layer_sizes[0].data, ds.layer_sizes[1].data), dims=['input', 'layer1'] )
ds['b0'] = xr.DataArray(np.random.randn(ds.layer_sizes[1].data), dims=['layer1'] )

ds['A1'] = xr.DataArray(np.random.randn(ds.layer_sizes[1].data, ds.layer_sizes[2].data), dims=['layer1', 'layer2'] )
ds['b1'] = xr.DataArray(np.random.randn(ds.layer_sizes[2].data), dims=['layer2'] )

ds['A2'] = xr.DataArray(np.random.randn(ds.layer_sizes[2].data, ds.layer_sizes[3].data), dims=['layer2', 'output'] )
ds['b2'] = xr.DataArray(np.random.randn(ds.layer_sizes[3].data), dims=['output'] )

In [91]:
ds.A0

<xarray.DataArray 'A0' (input: 2, layer1: 4)>
array([[ 1.76405235,  0.40015721,  0.97873798,  2.2408932 ],
       [ 1.86755799, -0.97727788,  0.95008842, -0.15135721]])
Dimensions without coordinates: input, layer1

In [92]:
ds.b2

<xarray.DataArray 'b2' (output: 2)>
array([-1.04855297, -1.42001794])
Dimensions without coordinates: output

In [94]:
ds.to_netcdf('/scratch/db194/mom6/tests_Phillips_2layer/Phillips_2layer_20km_ANN/ann_params.nc', mode='w')

In [95]:
xr.open_dataset('/scratch/db194/mom6/tests_Phillips_2layer/Phillips_2layer_20km_ANN/ann_params.nc')

<xarray.Dataset>
Dimensions:      (num_layers: 4, input: 2, layer1: 4, layer2: 4, output: 2)
Dimensions without coordinates: num_layers, input, layer1, layer2, output
Data variables:
    layer_sizes  (num_layers) int32 ...
    A0           (input, layer1) float64 ...
    b0           (layer1) float64 ...
    A1           (layer1, layer2) float64 ...
    b1           (layer2) float64 ...
    A2           (layer2, output) float64 ...
    b2           (output) float64 ...

In [96]:
# in Flax there are two parts (https://flax.readthedocs.io/en/latest/api_reference/flax.linen/_autosummary/flax.linen.Dense.html)
# - a part that evaluates the dense layer
# - a part that sets up the weights etc. 
def dense(A, b, x): 
    ''' dense layer
        y = xA + b
        Note : row vector format
    '''

    y = np.matmul(x, A) + b
    
    return y

def relu(x): 
    ''' relu evaluation (non-linear activation function) 
    '''
    y = np.maximum(x,0.)
    
    return y

In [97]:
x = np.array([6, 7]).astype('float32')

In [98]:
dense(ds.A0.data, ds.b0.data, x)

array([23.55400116, -4.02940341, 12.6670904 , 13.84013224])

In [99]:
y = np.zeros(4)

for j in range(4):
    for i in range(2):
        y[j] = y[j] + x[i] * ds.A0.data[i,j]
        
    y[j] = y[j] + ds.b0.data[j]         

In [100]:
y

array([23.55400116, -4.02940341, 12.6670904 , 13.84013224])

In [57]:
dense(ds.A2.data, ds.b2.data, dense(ds.A1.data, ds.b1.data, dense(ds.A0.data, ds.b0.data, x)))

array([38.11837683, 49.53725901])

In [101]:
relu(dense(ds.A2.data, ds.b2.data, relu(dense(ds.A1.data, ds.b1.data, relu(dense(ds.A0.data, ds.b0.data, x))) ) ) )

array([9.8018168, 0.       ])